In [ ]:
%pylab inline

In [ ]:
import sys
import os
import mmap

In [ ]:
pwd

In [ ]:
from multiprocessing import shared_memory

sys.path.append("..")
import extract_memory
layout=extract_memory.get_header_format('../memory_layout.h')
NAME='NW_SRC0_HDR'
shmem_main = shared_memory.SharedMemory(name=NAME)

layout_log=extract_memory.get_header_format('../layout_log.h')
NAME='NW_LOG'
#shm_log = shared_memory.SharedMemory(name=NAME, size=1024*1024)

layoutb=extract_memory.get_header_format('../layout_boxes.h')
shmem_boxes = shared_memory.SharedMemory(name="NW_BUFFER_BOXES")
buf_boxes=shmem_boxes.buf
#test

In [ ]:
nptypes={'double':np.float64, 'uint16_t':np.uint16,'uint8_t':np.uint8}
def shmem_item(shmem1,layout,varname):
    item_info=layout[1][varname]
    data=np.frombuffer(shmem1, dtype=nptypes[item_info['type']],
                       count=item_info['num_elements'], offset=item_info['bytenum_current'])
    if len(data)==1:
        data=data[0]
    return data

def shmem_write(shmem1,layout,varname,data):
    item_info=layout[1][varname]
    bytez =np.array(data, dtype=nptypes[item_info['type']]).tobytes()   
    offset=item_info['bytenum_current']
    shmem1[offset : offset + len(bytez)] = bytez

class NW_shmem:
    def __init__(self,mem,layout):
        self.mem = mem
        self.layout = layout

    def item(self,name):
        return shmem_item(self.mem, self.layout, name)

    def write(self,name,data):
        return shmem_write(self.mem, self.layout, name, data)

    def get_fields(self):
        return list( self.layout[1].keys() )
        
boxmem = NW_shmem(buf_boxes,layoutb)
num_boxes=boxmem.item('num_boxes')
#centroid_x=shmem_item( buf_boxes, layoutb, 'centroid_x')[0:num_boxes]
#centroid_y=shmem_item( buf_boxes, layoutb, 'centroid_y')[0:num_boxes]

reference_x=boxmem.item('reference_x')[0:num_boxes]
reference_y=boxmem.item('reference_y')[0:num_boxes]

mem0 = NW_shmem(shmem_main.buf,layout)

print( num_boxes )#, centroid_x[0],  centroid_y[0] )

In [ ]:
print( boxmem.get_fields() )

In [ ]:
#plt.plot( (cenx2 - cenx1) / (0.2+0.2) )

In [ ]:
num_mirs=97
mirs=np.zeros(num_mirs)
reps=3
positions=[-0.2,0.2]
centroids=np.zeros( (len(positions), 2, reps, len(mirs), num_boxes ) )
for npos,pos1 in enumerate(positions):
    for nrep in np.arange(reps):
        for nmirror in np.arange(num_mirs):
            mirs *= 0
            mirs[nmirror] = pos1
            boxmem.write("mirror_voltages",mirs)
            time.sleep(0.1)

            displ_x = boxmem.item('centroid_x')[0:num_boxes].copy() - reference_x # Force recompute
            displ_y = boxmem.item('centroid_y')[0:num_boxes].copy() - reference_y # Force recompute

            # Remove tip/tilt
            centroids[npos,0,nrep,nmirror] =    displ_x - np.mean( displ_x )
            centroids[npos,1,nrep,nmirror] = - (displ_y - np.mean( displ_y )  )

In [ ]:
# Average the 3 repeats, separate the two positions/directions
pos1=np.mean(centroids[0,:,:,:], axis=1 ) 
pos2=np.mean(centroids[1,:,:,:], axis=1 )

# Subtract positive from negative post
diff = (pos1-pos2) / (positions[0]-positions[1] ) # In pixels
diff = diff * boxmem.item('pixel_um') / boxmem.item('focal_um')
which_mirror = 48
plt.plot( diff[0,which_mirror]  )
plt.plot( diff[1,which_mirror]  )

# Reshape into a grid, then flatten and intersperse
x_uniq = np.sort( np.unique( reference_x ) )
y_uniq = np.sort( np.unique( reference_y ) )
size_big_grid = int( len(x_uniq) )

idx_to_grid = np.zeros( (2, len(reference_x) ), dtype='int' )
for nbox in np.arange(len(reference_x)):
    # Put y first so it's in array order
    idx_to_grid[0,nbox] = ( np.where( reference_y[nbox] == y_uniq )[0] )
    idx_to_grid[1,nbox] = ( np.where( reference_x[nbox] == x_uniq )[0] )

def circ_array_to_grid(infx, infy):
    grid2=np.zeros( (infx.shape[0], size_big_grid, size_big_grid, 2) )
    for nbox in np.arange(infx.shape[1]):
        grid2[:,idx_to_grid[0,nbox],idx_to_grid[1,nbox], 0] = infx[:,nbox]
        grid2[:,idx_to_grid[0,nbox],idx_to_grid[1,nbox], 1] = infy[:,nbox]
    grid2 = np.reshape( grid2, (infx.shape[0], 2 * size_big_grid**2 ) )
    return grid2
infl_mat=circ_array_to_grid( diff[0], diff[1] )
np.savetxt( "influence.txt", infl_mat )
np.savetxt( "/MiniWave/MiniWaveConfiguration/InfluenceMatrix_ALPAO_7.5mm_new.txt", infl_mat )

In [ ]:
nm=68
plt.figure(figsize=(15,4))
plt.plot( infl_mat[nm,::2] )
plt.plot( infl_mat[nm,1::2] )

In [ ]:
plt.figure(figsize=(15,4))
plt.imshow( infl_mat[:,::], cmap='bone' )

In [ ]:
plt.imshow( infl_mat[14,:,:,1] )
plt.colorbar()

##### infl_mat.shape

In [ ]:
infl_mat.shape

In [ ]:

influence_square = np.zeros( (num_mirs, x_uniq * y_uniq * 2 ) )

In [ ]:
diff.shape

In [ ]:
centroids.shape

In [ ]:
# x: Just first run of each
plt.plot( (centroids[0,0,0,which_mirror,:] -
           centroids[1,0,0,which_mirror,:])/(positions[0]-positions[1])
        * boxmem.item('pixel_um') / boxmem.item('focal_um'))

# y: Just first run of each
plt.plot( (centroids[0,1,0,which_mirror,:] -
           centroids[1,1,0,which_mirror,:])/(positions[0]-positions[1])
        * boxmem.item('pixel_um') / boxmem.item('focal_um'))

In [ ]:
im1=np.loadtxt("/MiniWave/MiniWaveConfiguration/InfluenceMatrix_ALPAO_6.5mm_.dat",skiprows=1)

In [ ]:
valids=np.where( np.sum(im1**2,0)>0 )[0]
print( valids.shape )

In [ ]:
valids

In [ ]:
mirs -= 0.05
print (mirs.mean() )
boxmem.write("mirror_voltages",mirs)

In [ ]:
import zernike_functions
zterms_full=np.load("../zterms_full.npy")

In [ ]:
zterms_full.shape, zterms_full.shape[1]/2

In [ ]:
slopes_est[1:len(slopes_est)].mean()

In [ ]:
mirs=np.loadtxt("../mirror_voltages.txt")
infi=np.load('../influ.npy')

In [ ]:
#slopes_est2.reshape( (len(slopes_est2)//2, 2) ).T

In [ ]:
boxmem.layout[1].keys()

In [ ]:
inf_full = np.loadtxt(
    "c:/MiniWave/MiniWaveConfiguration/InfluenceMatrix_ALPAO_7.2mm_.dat" , skiprows=1)

In [ ]:
sys.path.append("c:/code/nextwave")
import functions_grid
nvalid_boxes = boxmem.item('pupil_radius_um')/boxmem.item('box_um')*1000
valid_indices=functions_grid.find_indices(inf_full, nvalid_boxes)
print( len(valid_indices) )

inf_full = inf_full[:,valid_indices]

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
DESIRED_MODES=85

from matplotlib.ticker import ScalarFormatter
formatter = ScalarFormatter()
formatter.set_scientific(False) # Example: disable scientific notation
formatter.set_useOffset(False) # Example: disable offset

#if len(self.acts_outside)>0:
#    self.influence[self.acts_outside,:] = 0 # Zero them out, if any
U, s, V = np.linalg.svd(inf_full, full_matrices=True)
s_recip = 1/s
plt.figure(figsize=(15,4))
plt.bar(np.arange(len(s_recip)), s_recip/s_recip[0] )
plt.semilogy()
plt.gca().yaxis.set_major_formatter(formatter)
plt.grid()

if DESIRED_MODES < len(s):
    s_recip[DESIRED_MODES:] = 0
    
s_recip[ s<1e-10 ] = 0 # Set any tinies (or 0) to zero

influence_inv = ( (U * s_recip) @ V[0:97,:] ).T
#print( max_boxes, len(valid_boxes), self.influence.shape, s.max(), valid_boxes[0:10], RR.shape )

plt.plot( s_recip/s_recip[0], 'r-')

In [ ]:
influence_inv.shape, slope_reshape.shape

In [ ]:
GAIN=0.5
Z_TERMS = 65

shmir =shmem_item( buf_boxes, layoutb, 'mirror_voltages')[0:97]
print( shmir.max())

centroid_x=boxmem.item('centroid_x')[0:num_boxes]
centroid_y=boxmem.item('centroid_y')[0:num_boxes]
reference_x=boxmem.item('reference_x')[0:num_boxes]
reference_y=boxmem.item('reference_y')[0:num_boxes]

# This matches what is shown on the screen
spot_displace_x =   centroid_x - reference_x  
spot_displace_y =   centroid_y - reference_y

MULT=5
plt.plot( (reference_x,reference_x+spot_displace_x*MULT),
         (reference_y,reference_y+spot_displace_y*MULT), 'g-' )
plt.plot( reference_x,reference_y, 'k.' )
plt.gca().invert_yaxis()
plt.gca().axis('equal')

spot_displace_y -= spot_displace_y.mean()
spot_displace_x -= spot_displace_x.mean()
spot_displace_y = -spot_displace_y

slope = np.concatenate( (spot_displace_y, spot_displace_x)) * (boxmem.item("pixel_um")) /( boxmem.item("focal_um")/1000)
coeff=np.matmul(zterms_full,slope)
zernikes=coeff[zernike_functions.CVS_to_OSA_map[0:len(coeff)]] # Return value will is OSA

#Z_TERMS = 35
#zt_inv = pinv(zterms_full[0:Z_TERMS] )

zt_inv2 = pinv(zterms_full[0:Z_TERMS] )
slopes_est2 =np.matmul( zt_inv2, zernikes[zernike_functions.OSA_to_CVS_map[0:Z_TERMS]] )
slopes_est=slopes_est2

plt.figure(figsize=(15,2))
plt.bar( np.arange(len(zernikes[1:])), zernikes[1:] )

plt.figure(figsize=(15,4))
plt.plot( slopes_est2, label='full' )
plt.plot( slope, '.', label='orig' )
plt.plot( slopes_est, ls='--', label='partial' )
plt.xlim(0-5, len(slope)+5 )
plt.legend()

plt.figure(figsize=(15,4))
#plt.plot( mirs, 'o-' )
#plt.plot( shmir, 's-' )
plt.grid()

halfway = slopes_est.shape[0]//2
#slopes_desired1 = np.vstack( (-slopes_est[halfway:],slopes_est[:halfway]) ).T.flatten()
slope_reshape=np.concatenate( (slopes_est2[::2], slopes_est2[1::2] ) )
mirs_new = np.matmul(infi.T,slope_reshape/1000.0)
if False:
    plt.plot( mirs_new*GAIN, 'x' )

#slope_reshape = np.vstack( (slope[halfway:],slope[:halfway]) ).T.flatten()
#mirs_new = np.matmul(infi.T, slope_reshape/1000.0)
#plt.plot( -mirs_new*GAIN, '.' )
#
#slope_reshape = np.vstack( (slopes_est2[halfway:],slopes_est2[:halfway]) ).T.flatten()
#mirs_new = np.matmul(infi.T, slope_reshape/1000.0)
#plt.plot( -mirs_new*GAIN, 'k:' )

# Last one: one used
slope_reshape = np.vstack( (slopes_est2[halfway:],slopes_est2[:halfway]) ).T.flatten()
mirs_new = np.matmul(influence_inv.T, slope_reshape/1000.0)
plt.plot( -mirs_new*GAIN, 'r--' )

In [ ]:
mirs_now=boxmem.item("mirror_voltages")[0:97]
boxmem.write("mirror_voltages",-mirs_new*GAIN+mirs_now)
mem0.write("mode",[mem0.item("mode") | 0x20] )

In [ ]:
mem0.item("mode")

In [ ]:
mem0.mem.readonly

In [ ]:
plt.plot( -mirs_new*GAIN - shmir )

In [ ]:
plt.figure(figsize=(15,4))
slope_reshape = np.vstack( (slopes_est2[halfway:],slopes_est2[:halfway]) ).T.flatten()
slope_reshape = np.vstack( (slope[halfway:],slope[:halfway]) ).T.flatten()
plt.plot( slope_reshape/1000.0, label='est')

shlop=shmem_item( buf_boxes, layoutb, 'box_x_normalized')[0:num_boxes*2]
plt.plot( shlop, label='shmem')

plt.xlim(100,300)
plt.legend()